In [6]:
import httpx
import asyncio
from typing import Dict, Any
from pydantic import BaseModel

# Why use callables in Functional Programming?

The more I learn about Functional Programming (**FP**) the more I like it. I find it much code written in a more FP style much easier to debug and undertand. This post discusses something I first learned about in [Grokking Simplicity](): passing callables to a function, rather than calling the sub-function directly. I didn't understand why passing the callable is a much better approach, until I came across it myself.

TLDR: it's more modular, and easier to test.

To better explain, in my case I have a function to make a `GET` request and return the response and another function for error handling. Initially, here are those two functions.

In [9]:
from typing import TypedDict


async def get_url(
    url: str, client: httpx.AsyncClient, limiter: asyncio.Semaphore = None
) -> httpx.Response:
    """Send and async request to the given URL using the client

    Allows for the optimal use of a Semaphore to restrict simultaneous Async calls.

    Parameters
    ---------
    url : str
        the url to get
    client : httpx.AsyncClient
        the httpx client to use for the call
    limiter : asyncio.Semaphore
        the Semaphore instance to use. Optional
    
    Returns
    -------
    httpx.Response
        the reponse object from the `url`
    """
    assert isinstance(client, httpx.AsyncClient)
    "`client` must be an httpx.AsyncClient"
    if not limiter:
        response = await client.get(url)
    else:
        async with limiter:
            response = await client.get(url)
    response.raise_for_status()
    return response

class GetterArgs(TypedDict):
    url: str
    client: httpx.AsyncClient
    limiter: asyncio.Semaphore | None

async def get_response(
    getter_args: GetterArgs,
    retries: int = 10,
) -> httpx.Response:
    ''''''

    try:
        response: httpx.Response = await get_url(**getter_args)
    except httpx.HTTPStatusError as e:
        code = response.status_code
        if code == 429 or (code >= 100 and code < 200):
            if "try_count" not in locals():
                try_count = 1
            else:
                try_count += 1
            if try_count >= retries and retries >= 0:
                print("Max retries exceeded for 429 error")
                raise e
            response = await get_url(**getter_args)
        else:
            raise e
        # TODO: account for other error codes; e.g. redirections
    return response


You can see that I've defined `get_url` and `get_reponse`. I call `get_url` from `get_response` and this approach works just fine.

In [28]:
async with httpx.AsyncClient() as client:
    getter_args = GetterArgs(
        url='http://httpbin.org/get',
        client=client
    )
    response = await get_response(getter_args=getter_args)
assert response.status_code == 200
print('Success')

Success


Exactly as intended. The tricky part is testing `get_response`. 